In [2]:
import tweepy
import json

In [ ]:
# bryden's shit 
CONSUMER_KEY = 'aF03g35xEjOm5OQuFw8PSuwLF'
CONSUMER_SECRET ='5Ifz6t0UPa6BxaaVm7eTQF4cOe3XcpVsX535nFkaLqy5bp7ehg'
OAUTH_TOKEN = '594338908-f0JBTAwwVGoZErJ4f1YNAgkj2gws7FQaFur4r2gs'
OAUTH_TOKEN_SECRET = 'EOR9ptKEtAc0LyhsEOAxsyEshWdgTcJKzXS6iBln2iB6h'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET )

api = tweepy.API(auth)

# Example Query
q = "#DonaldTrump OR #HillaryClinton"

tweets_container = {}


# Cursor params are passed to the method (ex.api.search)
# Cursor will alow as to continue to retrieve tweets without worrying about hitting a page limit
#
# api.search arguments are:
#
#     :reference: https://dev.twitter.com/rest/reference/get/search/tweets
#     :allowed_param:'q', 'lang', 'locale', 'since_id', 'geocode',
#      'max_id', 'since', 'until', 'result_type', 'count',
#       'include_entities', 'from', 'to', 'source']
#
# To handle the rate limit:
# # http://docs.tweepy.org/en/v3.5.0/code_snippet.html#handling-the-rate-limit-using-cursors

def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError as e:
            print "Rate Limit Error %s" % e
            # can handle in any way we want
            time.sleep(15 * 60)

for tweet in limit_handled(tweepy.Cursor(api.search,
                           q=q,
                           count=100,
                           result_type="recent",
                           include_entities=True,
                           lang="en").items()):
    
    # converting tweet to json
    tweet_json = json.dumps(tweet._json)
    tweet_json = json.loads(tweet_json)
    tweets_container[tweet_json['id']] = tweet_json
    
    # Test how many tweets we can store
    size = len(tweets_container.keys())
    if size % 500 == 0:
        print size
        
print count

500
1000
1500
